In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
x_train_path = 'archive/X_test_sat6.csv'
y_train_path = 'archive/y_test_sat6.csv'

X_train = pd.read_csv(x_train_path)
y_train = pd.read_csv(y_train_path)


X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
print ('We have',X_train.shape[0],'examples and each example is a list of',X_train.shape[1],'numbers with',y_train.shape[1],'possible classifications.')

X_train_img = X_train.reshape([80999,28,28,4]).astype(float)
print (X_train_img.shape)

We have 80999 examples and each example is a list of 3136 numbers with 6 possible classifications.
(80999, 28, 28, 4)


In [3]:
y_train[0]

array([0, 0, 1, 0, 0, 0], dtype=int64)

In [4]:
print(y_train[0])

[0 0 1 0 0 0]


In [5]:
print(X_train[0])

[102 102 103 ...  85  76 130]


In [6]:
X_train = (X_train-X_train.mean())/X_train.std()

In [7]:
print(X_train[0])

[-0.18139558 -0.18139558 -0.16354242 ... -0.48489916 -0.64557752
  0.31849268]


In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1024, input_shape=(3136,), activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    
    tf.keras.layers.Dense(32,activation=tf.nn.relu),
    tf.keras.layers.Dense(6,activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              3212288   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_2 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 198       
Total params: 3,483,110
Trainable params: 3,483,110
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(X_train,y_train,batch_size=32, epochs=5, verbose=1, validation_split=0.1)

Epoch 1/5
2279/2279 [==============================] - 46s 20ms/step - loss: 0.2458 - accuracy: 0.9150 - val_loss: 0.1209 - val_accuracy: 0.9563
Epoch 2/5
2279/2279 [==============================] - 43s 19ms/step - loss: 0.1407 - accuracy: 0.9492 - val_loss: 0.1091 - val_accuracy: 0.9611
Epoch 3/5
2279/2279 [==============================] - 43s 19ms/step - loss: 0.1216 - accuracy: 0.9576 - val_loss: 0.1242 - val_accuracy: 0.9531
Epoch 4/5
2279/2279 [==============================] - 45s 20ms/step - loss: 0.1123 - accuracy: 0.9622 - val_loss: 0.1279 - val_accuracy: 0.9512
Epoch 5/5
2279/2279 [==============================] - 44s 19ms/step - loss: 0.1006 - accuracy: 0.9655 - val_loss: 0.0854 - val_accuracy: 0.9686


In [16]:
preds = model.predict(X_train[-1000:], verbose=1)
ix = 465 #Type a number between 0 and 999 inclusive
#Tells what the image is
print ('Prediction:\n{:.1f}% probability barren land,\n{:.1f}% probability trees,\n{:.1f}% probability grassland,\n{:.1f}% probability other\n'.format(preds[ix,0]*100,preds[ix,1]*100,preds[ix,2]*100,preds[ix,3]*100))

print ('Ground Truth: ',end='')
if y_train[80999-(1000-ix),0] == 1:
    print ('Barren Land')
elif y_train[80999-(1000-ix),1] == 1:
    print ('Trees')
elif y_train[80999-(1000-ix),2] == 1:
    print ('Grassland')
else:
    print ('Other')

32/32 [==============================] - 0s 3ms/step
Prediction:
0.0% probability barren land,
0.0% probability trees,
100.0% probability grassland,
0.0% probability other

Ground Truth: Grassland


In [17]:
model.save('SAT6.model')

INFO:tensorflow:Assets written to: SAT6.model\assets
